<a href="https://colab.research.google.com/github/fiofana/NLR-Coffea/blob/main/scripts/bioinfo_file_manipulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 19.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import csv
import re
import sys
from Bio import SeqIO
from pathlib import Path
from collections import defaultdict

#Fasta Sequence Counter

In [ ]:
sequence_file = "/content/genome-annotation-humblotiana-Augustus-protein-sequence.fasta"

In [ ]:
def count_sequences(fasta):
    with open(fasta, "r") as f:
        return sum(1 for line in f if line.startswith(">"))

total = count_sequences(sequence_file)
print(f"Total of seqeunces: {total}")

Total of seqeunces: 71658


#Filter Fasta Check

In [ ]:
fasta_file = "/content/Galaxy700-[Filter FASTA on data 699 and data 247_ FASTA sequences].fasta"
ids_file = "/content/Galaxy699-[canephora_ASM_nbarc_id.txt (modified)].txt"
output_file = "ids_not_found.txt"

In [ ]:
def parse_fasta_ids(fasta_path):
    with open(fasta_path, "r") as f:
        return set(line[1:].split()[0] for line in f if line.startswith(">"))

def read_ids_file(ids_path):
    with open(ids_path, "r") as f:
        return set(line.strip() for line in f)

In [ ]:
fasta_ids = parse_fasta_ids(fasta_file)
input_ids = read_ids_file(ids_file)

not_found = input_ids - fasta_ids
found = input_ids & fasta_ids

with open(output_file, "w") as out:
    for id_ in sorted(not_found):
        out.write(id_ + "\n")

print(f"Total of IDs:                        {len(input_ids)}")
print(f"Total of sequences in FASTA:         {len(fasta_ids)}")
print(f"IDs found in FASTA:                  {len(found)}")
print(f"IDs NOT found in FASTA:              {len(not_found)}")

<generator object read_ids_file.<locals>.<genexpr> at 0x789af2e61be0>
Total of IDs:                        576
Total of sequences in FASTA:         576
IDs found in FASTA:                  576
IDs NOT found in FASTA:              0


#Pfam Merger

In [ ]:
# Define file path
hmm_file = pd.read_csv('/content/Pfam Scan Results - arabica-geisha.csv')

In [ ]:
hmm_file.head()

,seq id,alignment start,alignment end,envelope start,envelope end,hmm acc,hmm name,type,hmm start,hmm end,hmm length,bit score,E-value,significance,clan,predicted active site residues,NLR?
0,CM084640.1.g351,84,175,83,218,PF18052.6,Rx_N,Domain,2,89,93,50.8,1.6e-13,1,No_clan,NaN,NLR
1,CM084640.1.g351,251,473,246,492,PF00931.27,NB-ARC,Domain,6,219,248,140.4,"7,00E-41",1,CL0023,NaN,NLR
2,CM084640.1.g376,10,96,9,115,PF18052.6,Rx_N,Domain,2,89,93,58.5,6.3e-16,1,No_clan,NaN,NLR
3,CM084640.1.g376,180,231,176,236,PF00931.27,NB-ARC,Domain,5,51,248,32.0,8.8e-08,1,CL0023,NaN,NLR
4,CM084640.1.g377,14,159,2,176,PF00931.27,NB-ARC,Domain,85,225,248,69.2,3.8e-19,1,CL0023,NaN,NLR


In [ ]:
hmm_file.columns

Index(['seq id', 'alignment start', 'alignment end', 'envelope start',
       'envelope end', 'hmm acc', 'hmm name', 'type', 'hmm start', 'hmm end',
       'hmm length', 'bit score', 'E-value', 'significance', 'clan',
       'predicted active site residues', 'NLR?'],
      dtype='object')

In [ ]:
# Filter rows where hmm name is 'NB-ARC'
nb_arc_rows = hmm_file[hmm_file['hmm name'] == 'NB-ARC']

#  Get the unique 'seq id' values where 'NB-ARC' is present
unique_seq_ids = nb_arc_rows['seq id'].unique()

#  Filter the original DataFrame by the unique 'seq id' values
filtered_by_seq_id = hmm_file[hmm_file['seq id'].isin(unique_seq_ids)]

#  Get all unique 'hmm name' values associated with those 'seq id'
nb_arc_unique_table = pd.DataFrame({
    'hmm acc': filtered_by_seq_id['hmm acc'].unique(),
    'hmm name': filtered_by_seq_id['hmm name'].unique()
})

nb_arc_unique_table = nb_arc_unique_table.sort_values(by='hmm name', ascending=True)
nb_arc_unique_table

,hmm acc,hmm name
5,PF00670.26,AdoHcyase_NAD
38,PF12796.12,Ank_2
26,PF02485.26,Branch
36,PF20160.4,C-JID
8,PF04818.18,CID
11,PF03108.20,DBD_Tnp_Mut
22,PF03478.23,DUF295
35,PF14111.11,DUF4283
40,PF04937.20,DUF659
23,PF05553.16,DUF761


In [ ]:
columns_to_drop = [
    'alignment start', 'alignment end', 'envelope start',
    'envelope end', 'hmm start', 'hmm end',
    'hmm length', 'bit score', 'E-value', 'significance'
]

hmm_file_dropped = hmm_file.drop(columns=columns_to_drop)

hmm_file_dropped.head()

,seq id,hmm acc,hmm name,type,clan,predicted active site residues,NLR?
0,CM084640.1.g351,PF18052.6,Rx_N,Domain,No_clan,NaN,NLR
1,CM084640.1.g351,PF00931.27,NB-ARC,Domain,CL0023,NaN,NLR
2,CM084640.1.g376,PF18052.6,Rx_N,Domain,No_clan,NaN,NLR
3,CM084640.1.g376,PF00931.27,NB-ARC,Domain,CL0023,NaN,NLR
4,CM084640.1.g377,PF00931.27,NB-ARC,Domain,CL0023,NaN,NLR


In [ ]:
# Group by 'seq_id' and aggregate information
aggregated_df = hmm_file.groupby('seq id').agg({
    'hmm acc': lambda x: ', '.join(x.unique()),
    'hmm name': lambda x: ', '.join(x.unique()),
    'type': lambda x: ', '.join(x.unique()),
    'clan': lambda x: ', '.join(x.unique()),
    'predicted active site residues': lambda x: ', '.join(map(str, x.unique()))  # Convert all to string
}).reset_index()

aggregated_df.head()

,seq id,hmm acc,hmm name,type,clan,predicted active site residues
0,CM084640.1.g1009,"PF18052.6, PF00931.27","Rx_N, NB-ARC",Domain,"No_clan, CL0023",nan
1,CM084640.1.g1075,"PF18052.6, PF00931.27","Rx_N, NB-ARC",Domain,"No_clan, CL0023",nan
2,CM084640.1.g1141,"PF18052.6, PF00931.27","Rx_N, NB-ARC",Domain,"No_clan, CL0023",nan
3,CM084640.1.g1155,"PF18052.6, PF00931.27","Rx_N, NB-ARC",Domain,"No_clan, CL0023",nan
4,CM084640.1.g1350,PF00931.27,NB-ARC,Domain,CL0023,nan


In [ ]:
# Opening a CSV file in write mode
with open('pfam-scan-arabica-geisha-merged.csv', 'w', newline='') as csv_file:
    writer = csv.writer(csv_file)

    # Write the header
    writer.writerow(aggregated_df.columns)

    # Write the data rows
    writer.writerows(aggregated_df.values)

# Non Aminoacid Filter

In [ ]:
input_file = "/content/intersect-eugenioides-ASM-filterfasta-Augustus-protein-sequences.fasta"
output_file = "eugenioides_ASM_sequences_no_X.fasta"

In [ ]:
total_seq = 0
filtered_seq = 0

# Filtra as sequências sem X
with open(output_file, "w") as out_f:
    for record in SeqIO.parse(input_file, "fasta"):
        total_seq += 1
        if "X" not in str(record.seq):
            filtered_seq += 1
            SeqIO.write(record, out_f, "fasta")

removed_seq = total_seq - filtered_seq

print("Arquivo salvo como:", output_file)
print("Total de sequências:", total_seq)
print("Sequências filtradas (sem X):", filtered_seq)
print("Sequências removidas (com X):", removed_seq)

#Coiled-coil Results Structure

In [ ]:
log_file = "/content/zx.txt"

with open(log_file) as f:
    log = f.read()

In [ ]:
genes = re.split(r"PEPCOIL of ", log)[1:]
total_genes = len(genes)
print(f"Total de genes detectados: {total_genes}")

data = []
genes_com_predicao = []
genes_sem_predicao = []

for g_index, g in enumerate(genes):
    lines = g.strip().splitlines()
    if not lines:
        continue

    gene_id = lines[0].strip()
    predicoes_gene = 0

    i = 0
    while i < len(lines):
        line = lines[i].strip()

        if line.startswith("probable coiled-coil from"):
            prev_line = lines[i - 1].strip() if i > 0 else ""
            next_line = lines[i + 1].strip() if i + 1 < len(lines) else ""

            match_frame = re.search(r"(\d+)\.\.(\d+)\s+frame\s+(\d+)\.\.(\d+)", prev_line)
            match_range = re.search(r"from (\d+) to (\d+)", line)
            match_score = re.search(r"score: ([\d\.]+).*?probability ([\d\.]+)", next_line)

            if match_range and match_score:
                start, end = match_range.groups()
                score, prob = match_score.groups()
                frame_start, frame_end = (None, None)

                if match_frame:
                    frame_start = match_frame.group(3)
                    frame_end = match_frame.group(4)

                data.append({
                    "Gene ID": gene_id,
                    "Início": int(start),
                    "Fim": int(end),
                    "Tamanho": int(end) - int(start) + 1,
                    "Score": float(score),
                    "Probabilidade": float(prob),
                    "Frame início": frame_start,
                    "Frame fim": frame_end
                })
                predicoes_gene += 1

        i += 1

    if predicoes_gene > 0:
        genes_com_predicao.append(gene_id)
    else:
        genes_sem_predicao.append(gene_id)

# Salva CSV
df = pd.DataFrame(data)
df.to_csv("coiled_coil_pepcoil_results.csv", index=False)

print(f"Total de genes analisados: {total_genes}")
print(f"Genes com predições: {len(genes_com_predicao)}")
print(f"Genes sem predições: {len(genes_sem_predicao)}")
print(f"Total de predições registradas: {len(df)}")

# Domain Filter

In [ ]:
fasta_file = "/content/intersect-humblotiana-filterfasta-Augustus-protein-sequences.fasta"
domains_file = "/content/NB-ARC - humblotiana.tsv"
output_file = "humblotiana_nbarc_domains.fasta"

In [ ]:
try:
    seq_dict = SeqIO.to_dict(SeqIO.parse(fasta_file, "fasta"))
except Exception as e:
    print(f"Erro ao ler o arquivo FASTA: {e}", file=sys.stderr)
    sys.exit(1)

In [ ]:
try:
    # Carrega as sequências FASTA
    seq_dict = SeqIO.to_dict(SeqIO.parse(fasta_file, "fasta"))

    # Abre o arquivo de saída
    with open(output_file, "w") as out_fasta, open(domains_file) as in_tsv:
        # Pula o cabeçalho do TSV
        next(in_tsv)

        for line_num, line in enumerate(in_tsv, 2):
            line = line.strip()
            if not line:
                continue

            try:
                fields = line.split("\t")
                if len(fields) < 6:
                    print(f"Linha {line_num} incompleta: {line}", file=sys.stderr)
                    continue

                seq_id = fields[0]
                try:
                    start = int(fields[1])  # alignment start
                    end = int(fields[2])    # alignment end
                except ValueError:
                    print(f"Linha {line_num}: valores de início/fim inválidos: {fields[1]}-{fields[2]}", file=sys.stderr)
                    continue

                pfam = fields[5]  # hmm acc

                if pfam == "PF00931.27":  # NB-ARC
                    if seq_id in seq_dict:
                        full_seq = seq_dict[seq_id].seq
                        domain_seq = full_seq[start-1:end]
                        # Escreve no arquivo de saída
                        out_fasta.write(f">{seq_id}|NB-ARC|{start}-{end}\n{domain_seq}\n")
                    else:
                        print(f"AVISO: {seq_id} não encontrada no FASTA", file=sys.stderr)

            except Exception as e:
                print(f"Erro na linha {line_num}: {e}", file=sys.stderr)

except Exception as e:
    print(f"Erro fatal: {e}", file=sys.stderr)
    sys.exit(1)

print(f"Resultados salvos em {output_file}", file=sys.stderr)

Resultados salvos em humblotiana_nbarc_domains.fasta


# iTOL

In [ ]:
chromosome_mapping = {
    # C. arabica
    "CM061576.1": "Ca_Bourbon_chr1c",
    "CM061577.1": "Ca_Bourbon_chr1e",
    "CM061578.1": "Ca_Bourbon_chr2c",
    "CM061579.1": "Ca_Bourbon_chr2e",
    "CM061580.1": "Ca_Bourbon_chr3c",
    "CM061581.1": "Ca_Bourbon_chr3e",
    "CM061582.1": "Ca_Bourbon_chr4c",
    "CM061583.1": "Ca_Bourbon_chr4e",
    "CM061584.1": "Ca_Bourbon_chr5e",
    "CM061585.1": "Ca_Bourbon_chr5c",
    "CM061586.1": "Ca_Bourbon_chr6c",
    "CM061587.1": "Ca_Bourbon_chr6e",
    "CM061588.1": "Ca_Bourbon_chr7c",
    "CM061589.1": "Ca_Bourbon_chr7e",
    "CM061590.1": "Ca_Bourbon_chr8e",
    "CM061591.1": "Ca_Bourbon_chr8c",
    "CM061592.1": "Ca_Bourbon_chr9c",
    "CM061593.1": "Ca_Bourbon_chr9e",
    "CM061594.1": "Ca_Bourbon_chr10e",
    "CM061595.1": "Ca_Bourbon_chr10c",
    "CM061596.1": "Ca_Bourbon_chr11c",
    "CM061597.1": "Ca_Bourbon_chr11e",

    "NC_039898.1": "Ca_Caturra_chr1c",
    "NC_039899.1": "Ca_Caturra_chr1e",
    "NC_039900.1": "Ca_Caturra_chr2c",
    "NC_039901.1": "Ca_Caturra_chr2e",
    "NC_039902.1": "Ca_Caturra_chr3c",
    "NC_039903.1": "Ca_Caturra_chr3e",
    "NC_039904.1": "Ca_Caturra_chr4c",
    "NC_039905.1": "Ca_Caturra_chr4e",
    "NC_039906.1": "Ca_Caturra_chr5e",
    "NC_039907.1": "Ca_Caturra_chr5c",
    "NC_039908.1": "Ca_Caturra_chr6c",
    "NC_039909.1": "Ca_Caturra_chr6e",
    "NC_039910.1": "Ca_Caturra_chr7c",
    "NC_039911.1": "Ca_Caturra_chr7e",
    "NC_039912.1": "Ca_Caturra_chr8e",
    "NC_039913.1": "Ca_Caturra_chr8c",
    "NC_039914.1": "Ca_Caturra_chr9c",
    "NC_039915.1": "Ca_Caturra_chr9e",
    "NC_039916.1": "Ca_Caturra_chr10e",
    "NC_039917.1": "Ca_Caturra_chr10c",
    "NC_039918.1": "Ca_Caturra_chr11c",
    "NC_039919.1": "Ca_Caturra_chr11e",

    "CM071891.1": "Ca_ET39HiFi_chr1c",
    "CM071892.1": "Ca_ET39HiFi_chr1e",
    "CM071893.1": "Ca_ET39HiFi_chr2c",
    "CM071894.1": "Ca_ET39HiFi_chr2e",
    "CM071895.1": "Ca_ET39HiFi_chr3c",
    "CM071896.1": "Ca_ET39HiFi_chr3e",
    "CM071897.1": "Ca_ET39HiFi_chr4c",
    "CM071898.1": "Ca_ET39HiFi_chr4e",
    "CM071899.1": "Ca_ET39HiFi_chr5e",
    "CM071900.1": "Ca_ET39HiFi_chr5c",
    "CM071901.1": "Ca_ET39HiFi_chr6c",
    "CM071902.1": "Ca_ET39HiFi_chr6e",
    "CM071903.1": "Ca_ET39HiFi_chr7c",
    "CM071904.1": "Ca_ET39HiFi_chr7e",
    "CM071905.1": "Ca_ET39HiFi_chr8e",
    "CM071906.1": "Ca_ET39HiFi_chr8c",
    "CM071907.1": "Ca_ET39HiFi_chr9c",
    "CM071908.1": "Ca_ET39HiFi_chr9e",
    "CM071909.1": "Ca_ET39HiFi_chr10e",
    "CM071910.1": "Ca_ET39HiFi_chr10c",
    "CM071911.1": "Ca_ET39HiFi_chr11c",
    "CM071912.1": "Ca_ET39HiFi_chr11e",

    "CM072026.1": "Ca_ET39v06_1c",
    "CM072027.1": "Ca_ET39v06_1e",
    "CM072028.1": "Ca_ET39v06_2c",
    "CM072029.1": "Ca_ET39v06_2e",
    "CM072030.1": "Ca_ET39v06_3c",
    "CM072031.1": "Ca_ET39v06_3e",
    "CM072032.1": "Ca_ET39v06_4c",
    "CM072033.1": "Ca_ET39v06_4e",
    "CM072034.1": "Ca_ET39v06_5e",
    "CM072035.1": "Ca_ET39v06_5c",
    "CM072036.1": "Ca_ET39v06_6c",
    "CM072037.1": "Ca_ET39v06_6e",
    "CM072038.1": "Ca_ET39v06_7c",
    "CM072039.1": "Ca_ET39v06_7e",
    "CM072040.1": "Ca_ET39v06_8e",
    "CM072041.1": "Ca_ET39v06_8c",
    "CM072042.1": "Ca_ET39v06_9c",
    "CM072043.1": "Ca_ET39v06_9e",
    "CM072044.1": "Ca_ET39v06_10e",
    "CM072045.1": "Ca_ET39v06_10c",
    "CM072046.1": "Ca_ET39v06_11c",
    "CM072047.1": "Ca_ET39v06_11e",

    "CM084640.1": "Ca_Geisha_chr1c",
    "CM084641.1": "Ca_Geisha_chr1e",
    "CM084642.1": "Ca_Geisha_chr2c",
    "CM084643.1": "Ca_Geisha_chr2e",
    "CM084644.1": "Ca_Geisha_chr3c",
    "CM084645.1": "Ca_Geisha_chr3e",
    "CM084646.1": "Ca_Geisha_chr4c",
    "CM084647.1": "Ca_Geisha_chr4e",
    "CM084648.1": "Ca_Geisha_chr5e",
    "CM084649.1": "Ca_Geisha_chr5c",
    "CM084650.1": "Ca_Geisha_chr6c",
    "CM084651.1": "Ca_Geisha_chr6e",
    "CM084652.1": "Ca_Geisha_chr7c",
    "CM084653.1": "Ca_Geisha_chr7e",
    "CM084654.1": "Ca_Geisha_chr8e",
    "CM084655.1": "Ca_Geisha_chr8c",
    "CM084656.1": "Ca_Geisha_chr9c",
    "CM084657.1": "Ca_Geisha_chr9e",
    "CM084658.1": "Ca_Geisha_chr10e",
    "CM084659.1": "Ca_Geisha_chr10c",
    "CM084660.1": "Ca_Geisha_chr11c",
    "CM084661.1": "Ca_Geisha_chr11e",

    # C. canephora (ASM e AUK)
    "CM071913.1": "Cc_ASM_chr1",
    "CM071914.1": "Cc_ASM_chr2",
    "CM071915.1": "Cc_ASM_chr3",
    "CM071916.1": "Cc_ASM_chr4",
    "CM071917.1": "Cc_ASM_chr5",
    "CM071918.1": "Cc_ASM_chr6",
    "CM071919.1": "Cc_ASM_chr7",
    "CM071920.1": "Cc_ASM_chr8",
    "CM071921.1": "Cc_ASM_chr9",
    "CM071922.1": "Cc_ASM_chr10",
    "CM071923.1": "Cc_ASM_chr11",

    "HG974428.1": "Cc_AUK_chr1",
    "HG974429.1": "Cc_AUK_chr2",
    "HG974430.1": "Cc_AUK_chr3",
    "HG974431.1": "Cc_AUK_chr4",
    "HG974432.1": "Cc_AUK_chr5",
    "HG974433.1": "Cc_AUK_chr6",
    "HG974434.1": "Cc_AUK_chr7",
    "HG974435.1": "Cc_AUK_chr8",
    "HG974436.1": "Cc_AUK_chr9",
    "HG974437.1": "Cc_AUK_chr10",
    "HG974438.1": "Cc_AUK_chr11",

    # C. eugenioides
    "CM071880.1": "Ce_ASM_chr1",
    "CM071881.1": "Ce_ASM_chr2",
    "CM071882.1": "Ce_ASM_chr3",
    "CM071883.1": "Ce_ASM_chr4",
    "CM071884.1": "Ce_ASM_chr5",
    "CM071885.1": "Ce_ASM_chr6",
    "CM071886.1": "Ce_ASM_chr7",
    "CM071887.1": "Ce_ASM_chr8",
    "CM071888.1": "Ce_ASM_chr9",
    "CM071889.1": "Ce_ASM_chr10",
    "CM071890.1": "Ce_ASM_chr11",

    "CM011091.1": "Ce_Ceug_chr1",
    "CM011092.1": "Ce_Ceug_chr2",
    "CM011093.1": "Ce_Ceug_chr3",
    "CM011094.1": "Ce_Ceug_chr4",
    "CM011095.1": "Ce_Ceug_chr5",
    "CM011096.1": "Ce_Ceug_chr6",
    "CM011097.1": "Ce_Ceug_chr7",
    "CM011098.1": "Ce_Ceug_chr8",
    "CM011099.1": "Ce_Ceug_chr9",
    "CM011100.1": "Ce_Ceug_chr10",
    "CM011101.1": "Ce_Ceug_chr11",

    # C. humblotiana
    "CM041208.1": "Ch_humblotiana_chr1",
    "CM041209.1": "Ch_humblotiana_chr2",
    "CM041210.1": "Ch_humblotiana_chr3",
    "CM041211.1": "Ch_humblotiana_chr4",
    "CM041212.1": "Ch_humblotiana_chr5",
    "CM041213.1": "Ch_humblotiana_chr6",
    "CM041214.1": "Ch_humblotiana_chr7",
    "CM041215.1": "Ch_humblotiana_chr8",
    "CM041216.1": "Ch_humblotiana_chr9",
    "CM041217.1": "Ch_humblotiana_chr10",
    "CM041218.1": "Ch_humblotiana_chr11"
}

In [ ]:
input_treefile = "/content/PREF.treefile"
output_treefile = "arvore_processada.nwk"

In [ ]:
scaffold_mapping = {
    r'^JARPVW': "Ca_Bourbon_scaffold",
    r'^NW_': "Ca_Caturra_scaffold",
    r'^JAZHGF': "Ca_ET39HiFi_scaffold",
    r'^JAZHSI': "Ca_ET39v06_scaffold",
    r'^JBELAZ': "Ca_Geisha_scaffold",

    r'^JAZHGG': "Cc_ASM_scaffold",
    r'^HG7': "Cc_AUK_scaffold",

    r'^JAZHGH': "Ce_ASM_scaffold",
    r'^RHJT': "Ce_Ceug_scaffold",

    r'^JACYVW': "Ch_Humblotiana_scaffold"
}

In [ ]:
def process_tree_file(input_file, output_file):
    with open(input_file, "r") as f:
        tree = f.read()

    pattern = re.compile(r'(\b[A-Z0-9_\.]+\b)(\.g\d+)(?:[\|\-].*?)?(\W|$)')

    def replacer(match):
        original_id = match.group(1)  # ID original
        suffix = match.group(2)      # .gXXXXX
        ending = match.group(3)      # Delimitador

        if original_id in chromosome_mapping:
            new_id = chromosome_mapping[original_id]
            return f"{new_id}{suffix}{ending}"

        for scaffold_prefix, scaffold_name in scaffold_mapping.items():
            if re.match(scaffold_prefix, original_id):
                return f"{scaffold_name}_{original_id}{suffix}{ending}"

        return f"{original_id}{suffix}{ending}"

    # Aplica a substituição
    processed_tree = pattern.sub(replacer, tree)

    with open(output_file, "w") as f:
        f.write(processed_tree)

    print(f"Arquivo processado e salvo em: {output_file}")

In [ ]:
import re

def process_tree_file(input_file, output_file):
    with open(input_file, "r") as f:
        tree = f.read()

    # Atualizado: captura o prefixo (ID do contig/cromossomo), .gXXXXX e ignora todo o resto após isso
    pattern = re.compile(r'(\b[A-Z0-9_\.]+)(\.g\d+)(?:[\|\-][^\s\)\(,]*)?')

    def replacer(match):
        original_id = match.group(1)  # Ex: CM011101.1 ou JARPVW...
        suffix = match.group(2)       # Ex: .g100989

        # 1. Mapeamento exato de cromossomos
        if original_id in chromosome_mapping:
            new_id = chromosome_mapping[original_id]
            return f"{new_id}{suffix}"

        # 2. Mapeamento por prefixo de scaffold
        for scaffold_prefix, scaffold_name in scaffold_mapping.items():
            if re.match(scaffold_prefix, original_id):
                return f"{scaffold_name}_{original_id}{suffix}"

        # 3. Se não for cromossomo nem scaffold, apenas limpa o sufixo
        return f"{original_id}{suffix}"

    # Substitui os nomes das folhas
    processed_tree = pattern.sub(replacer, tree)

    with open(output_file, "w") as f:
        f.write(processed_tree)

    print(f"✅ Arquivo processado e salvo em: {output_file}")

process_tree_file(input_treefile, output_treefile)

In [ ]:
# Cores por espécie
SPECIES_COLORS = {
    "Ca_": "#cc79a7",  # Arabica (roxo)
    "Cc_": "#e69f00",  # Canephora (laranja)
    "Ce_": "#f0e442",  # Eugenioides (amarelo)
    "Ch_": "#009e73"   # Humblotiana (verde)
}

def extract_ids_from_newick(newick_file):
    """Extrai todos os IDs únicos de um arquivo Newick"""
    with open(newick_file, 'r') as f:
        tree = f.read()

    # Encontra todos os IDs no formato "X123" ou "X123.g456"
    ids = re.findall(r'([A-Za-z][A-Za-z0-9_\.]+)', tree)
    return sorted(set(ids))  # Remove duplicados e ordena

def determine_color(taxon_id):
    """Determina a cor com base no prefixo do ID"""
    for prefix, color in SPECIES_COLORS.items():
        if taxon_id.startswith(prefix):
            return color
    return "#ffffff"  # Padrão branco se não reconhecer

def generate_itol_dataset(ids, output_file):
    """Gera o arquivo de dataset para o iTOL"""
    with open(output_file, 'w') as f:
        # Escreve o cabeçalho
        f.write("DATASET_COLORSTRIP\n")
        f.write("SEPARATOR SPACE\n")
        f.write("DATASET_LABEL Species\n")
        f.write("COLOR #ffffff\n\n")

        f.write("COLOR_BRANCHES 1\n\n")

        f.write("LEGEND_TITLE Species\n")
        f.write("LEGEND_SHAPES 1 1 1 1\n")
        f.write("LEGEND_COLORS #cc79a7 #e69f00 #f0e442 #009e73\n")
        f.write("LEGEND_LABELS Arabica Canephora Eugenioides Humblotiana\n\n")

        f.write("STRIP_WIDTH 25\n")
        f.write("SHOW_INTERNAL 0\n")
        f.write("SHOW_STRIP_LABELS 0\n\n")

        f.write("DATA\n")

        # Escreve cada ID com sua cor correspondente
        for taxon_id in ids:
            color = determine_color(taxon_id)
            f.write(f"{taxon_id} {color}\n")

# Processamento principal
if __name__ == "__main__":
    input_tree = "arvore_processada.nwk"
    output_dataset = "species_colors_itol.txt"

    print(f"Processando arquivo Newick: {input_tree}")
    all_ids = extract_ids_from_newick(input_tree)

    print(f"Gerando dataset para o iTOL: {output_dataset}")
    generate_itol_dataset(all_ids, output_dataset)

    print(f"Concluído! {len(all_ids)} IDs processados.")

In [ ]:
# Cores padronizadas para cromossomos (1c = 1e, 2c = 2e, etc.)
CHROMOSOME_COLORS = {
    "1": "#D9298A",
    "2": "#F25C05",
    "3": "#F29F05",
    "4": "#6393F2",
    "5": "#7663F2",
    "6": "#2811CB",
    "7": "#FF99FB",
    "8": "#FF8979",
    "9": "#FFE6A1",
    "10": "#90EE90",
    "11": "#A9A2DC",
}

def extract_chromosome_from_id(taxon_id):
    """Extrai o número do cromossomo (ex: 'chr1c' → '1', '2e' → '2')"""
    chrom_part = taxon_id.split("_")[-1].split(".")[0]  # Pega "chr1c" ou "2e"
    return re.sub(r'[^0-9]', '', chrom_part)  # Remove letras (c/e/chr)

def generate_itol_colorstrip(newick_file, output_file):
    with open(newick_file, 'r') as f:
        tree = f.read()

    # Extrai todos os IDs únicos da árvore
    taxon_ids = re.findall(r'([A-Za-z][A-Za-z0-9_\.]+\.g\d+)', tree)

    with open(output_file, 'w') as f:
        f.write("DATASET_COLORSTRIP\nSEPARATOR SPACE\n")
        f.write("DATASET_LABEL Chromosome_Colors\n")
        f.write("COLOR #ffffff\n")  # Cor de fundo (não afeta as faixas)
        f.write("COLOR_BRANCHES 1\n\n")  # Pinta ramos automaticamente
        f.write("DATA\n")

        for taxon_id in taxon_ids:
            chrom_number = extract_chromosome_from_id(taxon_id)
            color = CHROMOSOME_COLORS.get(chrom_number, "#CCCCCC")  # Cinza para não mapeados
            f.write(f"{taxon_id} {color}\n")

# Uso:
generate_itol_colorstrip("arvore_processada.nwk", "itol_chromosome_colors.txt")

In [ ]:
# Species configuration
SPECIES = {
    "Ca_": ("1", "#cc79a7", "Arabica"),    # Square
    "Cc_": ("2", "#e69f00", "Canephora"),  # Circle
    "Ce_": ("3", "#f0e442", "Eugenioides"),# Triangle
    "Ch_": ("4", "#009e73", "Humblotiana") # Star
}

def generate_itol_symbols(newick_file, output_file):
    """Generate iTOL-compliant species symbol dataset"""
    with open(newick_file) as f:
        tree = f.read()
    gene_ids = re.findall(r'([A-Za-z][A-Za-z0-9_\.]+\.g\d+)', tree)

    with open(output_file, 'w') as f:
        # Header with validated parameters
        f.write("DATASET_BINARY\nSEPARATOR COMMA\n")
        f.write("DATASET_LABEL,Species\n")
        f.write("COLOR,#000000\n")

        # Field definitions (one per species)
        f.write("FIELD_SHAPES,1,2,3,4\n")
        f.write("FIELD_COLORS,#cc79a7,#e69f00,#f0e442,#009e73\n")
        f.write("FIELD_LABELS,Arabica,Canephora,Eugenioides,Humblotiana\n\n")

        # Data section
        f.write("DATA\n")
        for gene_id in gene_ids:
            fields = ["-1"] * 4  # Initialize all fields as omitted (-1)
            for i, (prefix, (shape, color, label)) in enumerate(SPECIES.items()):
                if gene_id.startswith(prefix):
                    fields[i] = "1"  # Filled shape for matching species
                    break
            f.write(f"{gene_id},{','.join(fields)}\n")

# Usage
generate_itol_symbols("arvore_processada.nwk", "itol_species_symbols.txt")

# GFF Filter by IDs

In [ ]:
chromosome_mapping = {
    # C. arabica - Bourbon
    "CM061576.1": ("ca1", "c"), "CM061577.1": ("ca1", "e"),
    "CM061578.1": ("ca2", "c"), "CM061579.1": ("ca2", "e"),
    "CM061580.1": ("ca3", "c"), "CM061581.1": ("ca3", "e"),
    "CM061582.1": ("ca4", "c"), "CM061583.1": ("ca4", "e"),
    "CM061584.1": ("ca5", "e"), "CM061585.1": ("ca5", "c"),
    "CM061586.1": ("ca6", "c"), "CM061587.1": ("ca6", "e"),
    "CM061588.1": ("ca7", "c"), "CM061589.1": ("ca7", "e"),
    "CM061590.1": ("ca8", "e"), "CM061591.1": ("ca8", "c"),
    "CM061592.1": ("ca9", "c"), "CM061593.1": ("ca9", "e"),
    "CM061594.1": ("ca10", "e"), "CM061595.1": ("ca10", "c"),
    "CM061596.1": ("ca11", "c"), "CM061597.1": ("ca11", "e"),

    # C. arabica - Caturra
    "NC_039898.1": ("ca1", "c"), "NC_039899.1": ("ca1", "e"),
    "NC_039900.1": ("ca2", "c"), "NC_039901.1": ("ca2", "e"),
    "NC_039902.1": ("ca3", "c"), "NC_039903.1": ("ca3", "e"),
    "NC_039904.1": ("ca4", "c"), "NC_039905.1": ("ca4", "e"),
    "NC_039906.1": ("ca5", "e"), "NC_039907.1": ("ca5", "c"),
    "NC_039908.1": ("ca6", "c"), "NC_039909.1": ("ca6", "e"),
    "NC_039910.1": ("ca7", "c"), "NC_039911.1": ("ca7", "e"),
    "NC_039912.1": ("ca8", "e"), "NC_039913.1": ("ca8", "c"),
    "NC_039914.1": ("ca9", "c"), "NC_039915.1": ("ca9", "e"),
    "NC_039916.1": ("ca10", "e"), "NC_039917.1": ("ca10", "c"),
    "NC_039918.1": ("ca11", "c"), "NC_039919.1": ("ca11", "e"),

    # C. arabica - ET39HiFi
    "CM071891.1": ("ca1", "c"), "CM071892.1": ("ca1", "e"),
    "CM071893.1": ("ca2", "c"), "CM071894.1": ("ca2", "e"),
    "CM071895.1": ("ca3", "c"), "CM071896.1": ("ca3", "e"),
    "CM071897.1": ("ca4", "c"), "CM071898.1": ("ca4", "e"),
    "CM071899.1": ("ca5", "e"), "CM071900.1": ("ca5", "c"),
    "CM071901.1": ("ca6", "c"), "CM071902.1": ("ca6", "e"),
    "CM071903.1": ("ca7", "c"), "CM071904.1": ("ca7", "e"),
    "CM071905.1": ("ca8", "e"), "CM071906.1": ("ca8", "c"),
    "CM071907.1": ("ca9", "c"), "CM071908.1": ("ca9", "e"),
    "CM071909.1": ("ca10", "e"), "CM071910.1": ("ca10", "c"),
    "CM071911.1": ("ca11", "c"), "CM071912.1": ("ca11", "e"),

    # C. arabica - ET39v06
    "CM072026.1": ("ca1", "c"), "CM072027.1": ("ca1", "e"),
    "CM072028.1": ("ca2", "c"), "CM072029.1": ("ca2", "e"),
    "CM072030.1": ("ca3", "c"), "CM072031.1": ("ca3", "e"),
    "CM072032.1": ("ca4", "c"), "CM072033.1": ("ca4", "e"),
    "CM072034.1": ("ca5", "e"), "CM072035.1": ("ca5", "c"),
    "CM072036.1": ("ca6", "c"), "CM072037.1": ("ca6", "e"),
    "CM072038.1": ("ca7", "c"), "CM072039.1": ("ca7", "e"),
    "CM072040.1": ("ca8", "e"), "CM072041.1": ("ca8", "c"),
    "CM072042.1": ("ca9", "c"), "CM072043.1": ("ca9", "e"),
    "CM072044.1": ("ca10", "e"), "CM072045.1": ("ca10", "c"),
    "CM072046.1": ("ca11", "c"), "CM072047.1": ("ca11", "e"),

    # C. arabica - Geisha
    "CM084640.1": ("ca1", "c"), "CM084641.1": ("ca1", "e"),
    "CM084642.1": ("ca2", "c"), "CM084643.1": ("ca2", "e"),
    "CM084644.1": ("ca3", "c"), "CM084645.1": ("ca3", "e"),
    "CM084646.1": ("ca4", "c"), "CM084647.1": ("ca4", "e"),
    "CM084648.1": ("ca5", "e"), "CM084649.1": ("ca5", "c"),
    "CM084650.1": ("ca6", "c"), "CM084651.1": ("ca6", "e"),
    "CM084652.1": ("ca7", "c"), "CM084653.1": ("ca7", "e"),
    "CM084654.1": ("ca8", "e"), "CM084655.1": ("ca8", "c"),
    "CM084656.1": ("ca9", "c"), "CM084657.1": ("ca9", "e"),
    "CM084658.1": ("ca10", "e"), "CM084659.1": ("ca10", "c"),
    "CM084660.1": ("ca11", "c"), "CM084661.1": ("ca11", "e"),

    # C. canephora - ASM
    "CM071913.1": ("cc1", ""), "CM071914.1": ("cc2", ""),
    "CM071915.1": ("cc3", ""), "CM071916.1": ("cc4", ""),
    "CM071917.1": ("cc5", ""), "CM071918.1": ("cc6", ""),
    "CM071919.1": ("cc7", ""), "CM071920.1": ("cc8", ""),
    "CM071921.1": ("cc9", ""), "CM071922.1": ("cc10", ""),
    "CM071923.1": ("cc11", ""),

    # C. canephora - AUK
    "HG974428.1": ("cc1", ""), "HG974429.1": ("cc2", ""),
    "HG974430.1": ("cc3", ""), "HG974431.1": ("cc4", ""),
    "HG974432.1": ("cc5", ""), "HG974433.1": ("cc6", ""),
    "HG974434.1": ("cc7", ""), "HG974435.1": ("cc8", ""),
    "HG974436.1": ("cc9", ""), "HG974437.1": ("cc10", ""),
    "HG974438.1": ("cc11", ""),

    # C. eugenioides - ASM
    "CM071880.1": ("ce1", ""), "CM071881.1": ("ce2", ""),
    "CM071882.1": ("ce3", ""), "CM071883.1": ("ce4", ""),
    "CM071884.1": ("ce5", ""), "CM071885.1": ("ce6", ""),
    "CM071886.1": ("ce7", ""), "CM071887.1": ("ce8", ""),
    "CM071888.1": ("ce9", ""), "CM071889.1": ("ce10", ""),
    "CM071890.1": ("ce11", ""),

    # C. eugenioides - Ceug
    "CM011091.1": ("ce1", ""), "CM011092.1": ("ce2", ""),
    "CM011093.1": ("ce3", ""), "CM011094.1": ("ce4", ""),
    "CM011095.1": ("ce5", ""), "CM011096.1": ("ce6", ""),
    "CM011097.1": ("ce7", ""), "CM011098.1": ("ce8", ""),
    "CM011099.1": ("ce9", ""), "CM011100.1": ("ce10", ""),
    "CM011101.1": ("ce11", ""),

    # C. humblotiana
    "CM041208.1": ("ch1", ""), "CM041209.1": ("ch2", ""),
    "CM041210.1": ("ch3", ""), "CM041211.1": ("ch4", ""),
    "CM041212.1": ("ch5", ""), "CM041213.1": ("ch6", ""),
    "CM041214.1": ("ch7", ""), "CM041215.1": ("ch8", ""),
    "CM041216.1": ("ch9", ""), "CM041217.1": ("ch10", ""),
    "CM041218.1": ("ch11", "")
}

In [ ]:
def convert_gff_to_mcscanx(input_gff, output_gff, gene_list_output, subgenome_output, remove_duplicates=True, min_gene_length=300):
    """
    Converts standard GFF3 to MCScanX format with subgenome tracking and duplicate removal.

    Args:
        input_gff: Path to input GFF3 file
        output_gff: Path for MCScanX-formatted output
        gene_list_output: Path for gene ID list (for BLAST)
        subgenome_output: Path to save subgenome information
        remove_duplicates: Whether to remove duplicate gene entries (default: True)
        min_gene_length: Minimum gene length to keep (default: 300bp)
    """
    # Read GFF3, skipping comments
    df = pd.read_csv(input_gff, sep='\t', comment='#', header=None,
                     names=["Seqid","Source","Type","Start","End","Score",
                            "Strand","Phase","Attributes"])

    # Filter only gene entries and calculate gene length
    genes = df[df['Type'] == 'gene'].copy()
    genes['Length'] = genes['End'] - genes['Start'] + 1

    # Filter by minimum gene length
    genes = genes[genes['Length'] >= min_gene_length]

    # Extract gene IDs from Attributes
    genes['GeneID'] = genes['Attributes'].str.extract(r'ID=([^;]+)')

    # Map chromosomes and track subgenomes
    genes['Chrom'] = genes['Seqid'].map(lambda x: chromosome_mapping.get(x, ("unknown", ""))[0])
    genes['Subgenome'] = genes['Seqid'].map(lambda x: chromosome_mapping.get(x, ("", ""))[1])

    # Filter out unknown chromosomes
    genes = genes[genes['Chrom'] != "unknown"]

    # Remove duplicate genes (same GeneID or overlapping coordinates)
    if remove_duplicates:
        # Track duplicates by GeneID
        duplicate_counts = genes['GeneID'].value_counts()
        duplicate_genes = duplicate_counts[duplicate_counts > 1].index.tolist()

        if duplicate_genes:
            print(f"Found {len(duplicate_genes)} duplicate gene IDs. Keeping first occurrence.")
            genes = genes.drop_duplicates(subset=['GeneID'], keep='first')

        # Track duplicates by genomic position (same chromosome and overlapping coordinates)
        coord_duplicates = genes.duplicated(subset=['Chrom', 'Start', 'End'], keep='first')
        if coord_duplicates.any():
            print(f"Removing {coord_duplicates.sum()} genes with duplicate coordinates.")
            genes = genes[~coord_duplicates]

    # Create MCScanX format (Chrom, GeneID, Start, End)
    mcscanx_gff = genes[['Chrom', 'GeneID', 'Start', 'End']]

    # Save outputs
    mcscanx_gff.to_csv(output_gff, sep='\t', index=False, header=False)
    genes[['GeneID']].to_csv(gene_list_output, index=False, header=False)

    # Enhanced subgenome info with additional metadata
    subgenome_info = genes[['GeneID', 'Subgenome', 'Seqid', 'Length']]
    subgenome_info.to_csv(subgenome_output, index=False, header=True)

    # Generate summary statistics
    summary_stats = {
        'total_genes': len(genes),
        'unique_genes': genes['GeneID'].nunique(),
        'avg_gene_length': genes['Length'].mean(),
        'subgenome_counts': genes['Subgenome'].value_counts().to_dict()
    }

    print(f"Conversion complete. Summary:\n{summary_stats}")
    print(f"MCScanX GFF saved to {output_gff}")
    print(f"Gene IDs for BLAST saved to {gene_list_output}")
    print(f"Subgenome information saved to {subgenome_output}")

def prepare_blast_db(fasta_file, gene_list, output_blast_db, evalue=1e-10, max_target_seqs=5):
    """
    Enhanced BLAST preparation with parameter controls.
    """
    blast_commands = f"""
    # Step 1: Extract protein sequences
    seqkit grep -f {gene_list} {fasta_file} > NLR_proteins.fasta

    # Step 2: Create BLAST database
    makeblastdb -in NLR_proteins.fasta -dbtype prot -out {output_blast_db}

    # Step 3: Run BLASTP (customizable parameters)
    blastp -query NLR_proteins.fasta \\
           -db {output_blast_db} \\
           -out NLR_blast_results.blast \\
           -evalue {evalue} \\
           -num_threads 4 \\
           -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore" \\
           -max_target_seqs {max_target_seqs}

    # Optional: Filter for high-quality hits
    awk '($3 >= 80 && $4 >= 100) {{print}}' NLR_blast_results.blast > NLR_blast_filtered.blast
    """

    print("Run these commands in your terminal:\n" + blast_commands)
    with open("run_blast_pipeline.sh", "w") as f:
        f.write(blast_commands)
    print("BLAST pipeline commands saved to 'run_blast_pipeline.sh'")

# Usage example with enhanced parameters:
convert_gff_to_mcscanx(
    input_gff="/content/eugenioides_ASM_nbarc.gff3",
    output_gff="mcscanx_eugenioides.gff",
    gene_list_output="gene_ids_for_blast.txt",
    subgenome_output="subgenome_info.csv",
    remove_duplicates=True,
    min_gene_length=300
)

prepare_blast_db(
    fasta_file="your_protein_sequences.fasta",
    gene_list="gene_ids_for_blast.txt",
    output_blast_db="NLR_blast_db",
    evalue=1e-20,  # More stringent e-value
    max_target_seqs=3  # Fewer targets for cleaner results
)

Found 1 duplicate gene IDs. Keeping first occurrence.
Conversion complete. Summary:
{'total_genes': 595, 'unique_genes': 595, 'avg_gene_length': np.float64(3907.657142857143), 'subgenome_counts': {'': 595}}
MCScanX GFF saved to mcscanx_eugenioides.gff
Gene IDs for BLAST saved to gene_ids_for_blast.txt
Subgenome information saved to subgenome_info.csv
Run these commands in your terminal:

    # Step 1: Extract protein sequences
    seqkit grep -f gene_ids_for_blast.txt your_protein_sequences.fasta > NLR_proteins.fasta

    # Step 2: Create BLAST database
    makeblastdb -in NLR_proteins.fasta -dbtype prot -out NLR_blast_db

    # Step 3: Run BLASTP (customizable parameters)
    blastp -query NLR_proteins.fasta \
           -db NLR_blast_db \
           -out NLR_blast_results.blast \
           -evalue 1e-20 \
           -num_threads 4 \
           -outfmt "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore" \
           -max_target_seqs 3

    # Optio

# GFF to BEDTrack

In [ ]:
def process_gff(gff_path, output_main, output_subgenome, feature_type, window_size):
    """
    Processa GFF gerando:
    - Arquivo principal: nomes básicos (ex: ca10), valores positivos
    - Arquivo subgenoma: nomes básicos, valores positivos (c) ou negativos (e)
    """
    density_main = defaultdict(int)
    density_subgenome = defaultdict(int)
    scaffolds_ignored = set()

    with open(gff_path, 'r') as gff:
        for line in gff:
            if line.startswith('#'):
                continue

            fields = line.strip().split('\t')
            if len(fields) < 9 or fields[2] != feature_type:
                continue

            original_chrom = fields[0]

            # Pular scaffolds JAZ
            if "JAZ" in original_chrom:
                scaffolds_ignored.add(original_chrom)
                continue

            # Aplicar mapeamento
            if original_chrom in chromosome_mapping:
                chrom_prefix, subgenome = chromosome_mapping[original_chrom]
                mapped_chrom = chrom_prefix  # Sem 'c' ou 'e'
            else:
                mapped_chrom = original_chrom
                subgenome = ""
                print(f"Aviso: Cromossomo {original_chrom} não mapeado")

            start = int(fields[3])
            end = int(fields[4])

            # Calcular janelas
            first_window = start // window_size
            last_window = end // window_size

            for window_num in range(first_window, last_window + 1):
                window_start = window_num * window_size
                window_end = window_start + window_size

                intersect_start = max(start, window_start)
                intersect_end = min(end, window_end)

                if intersect_start < intersect_end:
                    # Arquivo principal: sempre positivo
                    density_main[(mapped_chrom, window_start, window_end)] += 1

                    # Arquivo subgenoma: +valor para 'c', -valor para 'e'
                    if subgenome == 'c':
                        density_subgenome[(mapped_chrom, window_start, window_end)] += 1
                    elif subgenome == 'e':
                        density_subgenome[(mapped_chrom, window_start, window_end)] -= 1

    # Escrever arquivos
    with open(output_main, 'w') as f_main:
        for (chrom, start, end), count in sorted(density_main.items()):
            f_main.write(f"{chrom}\t{start}\t{end}\t{count}\n")

    with open(output_subgenome, 'w') as f_sub:
        for (chrom, start, end), count in sorted(density_subgenome.items()):
            f_sub.write(f"{chrom}\t{start}\t{end}\t{count}\n")

    print(f"Ignorados {len(scaffolds_ignored)} scaffolds")

In [ ]:
process_gff(
    gff_path="/content/coffea-all-nlr.gff3",
    output_main="nlrs_density_main.bedgraph",
    output_subgenome="nlrs_density_subgenome.bedgraph",
    feature_type="gene",
    window_size=700000
)

Ignorados 42 scaffolds


In [ ]:
from collections import defaultdict

def process_mcscanx_gff_coords(gff_path, output_main, window_size=700000):
    """
    Processa arquivo GFF do MCScanX e gera BedGraph com densidade em janelas de 700kb

    Parâmetros:
        gff_path (str): Caminho do arquivo GFF
        output_main (str): Caminho de saída BedGraph
        window_size (int): Tamanho da janela em bp (padrão: 700000)
    """
    # Dicionário para contar genes por janela
    window_counts = defaultdict(int)

    with open(gff_path, 'r') as gff:
        for line in gff:
            if line.startswith('#'):
                continue

            fields = line.strip().split('\t')
            if len(fields) < 4:
                continue

            chrom = fields[0]
            start = int(fields[2])
            end = int(fields[3])

            # Determinar as janelas que este gene intersecta
            first_window = start // window_size
            last_window = end // window_size

            for window_num in range(first_window, last_window + 1):
                window_start = window_num * window_size
                window_end = window_start + window_size

                # Calcula a interseção real
                intersect_start = max(start, window_start)
                intersect_end = min(end, window_end)

                if intersect_start < intersect_end:
                    window_key = (chrom, window_start, window_end)
                    window_counts[window_key] += 1

    # Escrever arquivo BedGraph ordenado
    with open(output_main, 'w') as f_main:
        for (chrom, start, end), count in sorted(window_counts.items()):
            f_main.write(f"{chrom}\t{start}\t{end}\t{count}\n")

# Uso:
process_mcscanx_gff_coords(
    gff_path="/content/mcscanx_coffea.gff",
    output_main="nlrs_density.bed",
    window_size=700000
)

# GFF to TBTools Gene Location

In [3]:
def parse_gff(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:

        for line in infile:
            # Pula linhas de comentário
            if line.startswith('#'):
                continue

            parts = line.strip().split('\t')
            if len(parts) < 9:
                continue

            # Verifica se é uma linha de gene
            if parts[2] == 'gene':
                chrname = parts[0]
                startpos = parts[3]
                endpos = parts[4]

                # Extrai o nome do gene do campo de atributos
                attributes = parts[8]
                gene_match = re.search(r'ID=([^;]+)', attributes)
                if gene_match:
                    genename = gene_match.group(1)
                else:
                    genename = "unknown"

                # Escreve a linha de saída
                outfile.write(f"{genename}\t{chrname}\t{startpos}\t{endpos}\n")

# Exemplo de uso:
input_gff = "/content/intersect-arabica-bourbon.gff3"  # Substitua pelo nome do seu arquivo de entrada
output_file = "genes_positions.tsv"  # Nome do arquivo de saída

parse_gff(input_gff, output_file)
print(f"Arquivo processado. Resultados salvos em {output_file}")

Arquivo processado. Resultados salvos em genes_positions.tsv
